In [16]:
#import the necessary packages 
#install dlib 
try:
    from scipy.spatial import distance
    from imutils import face_utils
    import imutils
    import dlib
    import cv2
    import playsound
    #this function here is used to calculate the euclidean distance
    def eye_aspect_ratio(eye):
        A = distance.euclidean(eye[1], eye[5])
        B = distance.euclidean(eye[2], eye[4])
        C = distance.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear

    #this function is used to play the alarm sound when the driver sleeps
    def sound_alarm(path):
        playsound.playsound(path)
    #thresh here is defined to compare with the euclidean distance whwn eyes are closed
    thresh = 0.25
    #frame check is to keep a counter to see if its a blink or if the driver is drowsy
    frame_check = 20
    detect = dlib.get_frontal_face_detector()
    predict = dlib.shape_predictor(r"C:\Users\ARF\AppData\Local\Programs\Python\Python37\Lib\site-packages\shape_detection\shape_predictor_68_face_landmarks.dat")# Dat file is the crux of the code

    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]
    #print((lStart,lEnd))
    cap=cv2.VideoCapture(0)
    flag=0
    while True:
        ret, frame=cap.read()
        frame = imutils.resize(frame, width=450)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        subjects = detect(gray, 0)
        for subject in subjects:
            shape = predict(gray, subject)
            shape = face_utils.shape_to_np(shape)#converting to NumPy Array
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
            if ear < thresh:
                flag += 1
                if flag >= frame_check:
                    sound_alarm(r'C:\music\beep.mp3')
            else:
                flag = 0
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            cv2.destroyAllWindows()
            cap.release()
            break
except KeyboardInterrupt:
    print("exit")

exit


In [17]:
#run to stop close the webcam
cap.release()